# Integration
* Simple/Linear
* Trapezoial
* Simpson
* Simpson 3/8
* Bode


* Inbuilt packages - For error-checking:
    * `scipy.integrate.trapz(y, x=None, dx=1.0, axis=-1)` or `np.trapz()`
    * `scipy.integrate.simps(y, x=None, dx=1, axis=-1, even='avg')` NOT PRESENT IN NUMPY
    * `scipy.integrate.quad(func, a, b, args=(), full_output=0, epsabs=1.49e-08, epsrel=1.49e-08, limit=50, points=None, weight=None, wvar=None, wopts=None, maxp1=50, limlst=50)` - Takes function and limits as input -  Returns absolute error, as well
    * `scipy.integrate.newton_cotes(rn, equal=0)` - Returns Coefficients/Weights of the Newton-Cotes Series, that can be weighted-summed (over functional values) linearly - Integration Order = N | (0 to N)
    * Others: `scipy.integrate.dblquad` or `scipy.integrate.tplquad`.

### Notes:

* Indexing from $0$ to $N(-1)$ (Array Indexing).

In [3]:
# Basic Imports
import numpy as np,\
       scipy as sc,\
       matplotlib.pyplot as plt

## Simple/Linear
$$I_{Lin} = \sum_{i = 1}^N {f(x_i)}\Delta x$$

In [11]:
def lin_int(n0, n1, func, step=1e-3):
    """
    INPUT:
    n0: Lower Limit
    n1: Upper Limit
    func: Function, to be integrated
    step: Step-size
    OUTPUT:
    I: Integral Value
    """
    N = np.ceil((n1-n0)/step) # Number of Slices
    x = np.linspace(n0, n1, N, dtype=float)
    f = np.array(func(x), dtype=float) # Array to store functional values
        
    # To store the integral value
    I = np.sum(f*step)

    return I

# Newton-Cotes Quadrature Rules

* A group of formulae, for numerical integration, based on evaluating the integrand at equally spaced points.

* For example,
    * Trapezoial
    * Simpson
    * Simpson 3/8
    * Bode

## Trapezoial (Extended)

$$I_{Trap} = \left(\frac{f(x_0) + f(x_N)}{2} + \sum_{i = 1}^{N-1} {f(x_i)}\right)\Delta x + \mathcal{O}(\Delta x^2f'')$$

In [5]:
def trap_int(n0, n1, func, step=1e-3):
    """
    INPUT:
    n0: Lower Limit
    n1: Upper Limit
    func: Function, to be integrated
    step: Step-size
    OUTPUT:
    I: Integral Value
    """
    N = np.ceil((n1-n0)/step) # Number of Slices
    x = np.linspace(n0, n1, N, dtype=float)
    f = np.array(func(x), dtype=float) # Array to store functional values
        
    # To store the integral value
    I = ((f[0] + f[-1])/2 + np.sum(f[1:-1]))*step

    return I

## Simpson (Extended)

For even $N$: $I_{Simps} = f(x_0) + f(x_N) + \sum_{i = 1}^{N-2}\frac{\Delta x}{3}\left(4f(x_{i}) + 2f(x_{i+1})\right) + \mathcal{O}(\Delta x^4f^{(4)})$

For odd $N$: $I_{Simps} \:\:+\!= \:I_{Correction}$

Where, $I_{Correction} = \frac{\Delta x}{12}\left[5f(x_N) + 8f(x_{N-1}) - f(x_{N-2})\right]$

### Notes

* The non-extended formula has (1, 4, 1) weights, with error as: $\mathcal{O}(\Delta x^5f^{(4)})$.

In [10]:
def simps_int(n0, n1, func, step=1e-3):
    """
    INPUT:
    n0: Lower Limit
    n1: Upper Limit
    func: Function, to be integrated
    step: Step-size
    OUTPUT:
    I: Integral Value
    """
    N = np.ceil((n1-n0)/step) # Number of Slices
    x = np.linspace(n0, n1, N, dtype=float)
    f = np.array(func(x), dtype=float) # Array to store functional values
    
    # To store the integral value
    I = 0.
    # For Even N
    for i in range(0, int(N)):
        if i == 0 or i == N-1:
            I += f[i]
        elif i%2: # i = [1, 3, 5, 7,..., N-3]
            I += 4*f[i]
        else: # i = [2, 4, 6, 8,..., N-2]
            I += 2*f[i]
    
    I *= (step/3)
    
    if N%2: # Odd N
        I_corr = (step/12)*(5*f[-1] + 8*f[-2] - f[-3])
        I += I_corr
    
    return I

## Simpson 3/8

$$I_{Simps3/8} = \left(\sum_{i = 0}^{N-3} {3f(x_i) + 9f(x_{i+1}) + 9f(x_{i+2}) + 3f(x_{i+3})}\right)\frac{\Delta x}{8} + \mathcal{O}(\Delta x^5f^{(4)})$$

In [9]:
def simps_3_8_int(n0, n1, func, step=1e-3):
    """
    INPUT:
    n0: Lower Limit
    n1: Upper Limit
    func: Function, to be integrated
    step: Step-size
    OUTPUT:
    I: Integral Value
    """
    N = np.ceil((n1-n0)/step) # Number of Slices
    x = np.linspace(n0, n1, N, dtype=float)
    f = np.array(func(x), dtype=float) # Array to store functional values
    
    # Manipulating f, in order to make it suitable for later summation
    # Padding with 0s
    f_app = f
    mod = N%4
    if mod != 0:
        f_app = np.append(f, np.zeros(int(4-mod)))
    
    # To store the integral value
    I = 0.
    for i in range(0, f_app.size-3, 3):
        I += 3*f_app[i] + 9*f_app[i+1] + 9*f_app[i+2] + 3*f_app[i+3]
    
    I *= (step/8)
    
    return I

## Bode

$$I_{Bode} = \left(\sum_{i = 0}^{N-4} {14f(x_i) + 64f(x_{i+1}) + 24f(x_{i+2}) + 64f(x_{i+3}) + 14f(x_{i+4})}\right)\frac{\Delta x}{45} + \mathcal{O}(\Delta x^7f^{(6)})$$

In [5]:
def bode_int(n0, n1, func, step=1e-3):
    """
    INPUT:
    n0: Lower Limit
    n1: Upper Limit
    func: Function, to be integrated
    step: Step-size
    OUTPUT:
    I: Integral Value
    """
    N = np.ceil((n1-n0)/step) # Number of Slices
    x = np.linspace(n0, n1, N, dtype=float)
    f = np.array(func(x), dtype=float) # Array to store functional values
    
    # Manipulating f, in order to make it suitable for later summation
    # Padding with 0s
    f_app = f
    mod = N%5
    if mod != 0:
        f_app = np.append(f, np.zeros(int(5-mod)))
    
    # To store the integral value
    I = 0.
    for i in range(0, f_app.size-4, 4):
        I += 14*f_app[i] + 64*f_app[i+1] + 24*f_app[i+2] + 64*f_app[i+3] + 14*f_app[i+4]
    
    I *= (step/45)
    
    return I

## For Testing Purposes

In [13]:
# Parameters
a, b = 0, 1
step = 1e-3 # Default is 1e-3

# In-Function Variables
N = np.ceil((b-a)/step) # Number of Slices
# print(N)

# Function Definitions
lin = lambda x: x
quad = lambda x: x**2
cubic = lambda x: x**3
biquad = lambda x: x**4
sin = lambda x: np.sin(x)
cos= lambda x: np.cos(x)
tan = lambda x: np.tan(x)
sqrt = lambda x: np.sqrt(x)


func = lambda x: np.exp(x)
x = np.linspace(a, b, N)
f = np.array(func(x), dtype=float)

# print(lin_int(a, b, step=1e-3, func=cos)) # Works with func = cos | 0 to PI | I = 0 (EXACT)
# print(trap_int(a, b, step=1e-3, func=cos)) # Doesn't give correct result for func = cos | 0 to PI | I = 0 
print(simps_int(0, 1, step=1e-4, func=lambda x: np.exp(x)))  # Works (kinda) with I = 0 (e.g. func = cos | 0 to PI) | I = 0 
print(simps_3_8_int(0, 1, step=1e-4, func=lambda x: np.exp(x))) # Doesn't give correct result for func = cos | 0 to PI | I = 0 
print(bode_int(0, 1, step=1e-4, func=lambda x: np.exp(x))) # Works (kinda) with func = cos | 0 to PI | I = 0



from scipy import integrate as inte
# print(inte.trapz(f, x, dx=step)) # In-built Trapezoidal - EXACT
# print(inte.simps(f, x, dx=step)) # In-built Simpson - NOT EXACT
# print(inte.quad(func, a, b)) # In-built Quadrature from FORTRAN's QUADPACK - Returns absolute error, as well - NOT EXACT

# In-built Netwon-Cotes - Returns Coefficients of the Series - EXACT
# exact = 2
# for N in [2, 4, 6, 8, 10]:
#    x = np.linspace(a, b, N + 1)
#    an, B = inte.newton_cotes(N, 1)
#    dx = (b - a) / N
#    quad = dx * np.sum(an * func(x))
#    error = abs(quad - exact)
#    print('{:2d}  {:10.9f}  {:.5e}'.format(N, quad, error))

1.7180193954128475
1.7181100002762
1.7172946380503402


## TIDBITS

In [ ]:
# Manipulating f, in order to make it suitable for later summation
f_appended = f
mod = N%3
if mod != 0:
    f_appended = np.append(f, np.zeros(int(3-mod)))

f_reshaped = np.reshape(f_appended, (int(np.ceil(N/3)), 3))

# Weights for summation
weights = np.array([1, 4, 2], dtype=float)
sum_vec = np.dot(f_reshaped, weights) 

I = (step/3)*np.sum(sum_vec)